In [1]:
import numpy as np
import scipy.integrate as integrate
import pandas as pd
import matplotlib.pyplot as plt
from financepy.models.black import *
from financepy.utils.global_types import *
import MJD as mjd
import time

%load_ext autoreload
%autoreload 2

####################################################################
#  FINANCEPY BETA Version 0.370 - This build: 28 Oct 2024 at 20:26 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [2]:
def carr_madan_integrand(cf, logk, u, alpha):
    top = cf(complex(u, -(alpha+1.0))) * np.exp(complex(0.0, -u*logk))
    bottom = complex(alpha*alpha + alpha - u*u, (2.0*alpha + 1.0)*u)
    result = top/bottom
    return result.real 

def MDJ_cf(forward, vol, a, b, lam, t, u):
    lnf_factor = complex(0, u*np.log(forward))
    jump_compensation = lam*(np.exp(a+b*b/2)-1)    
    drift_and_diffusion_term = complex(-0.5*u*u*vol*vol, u*(-0.5*vol*vol - jump_compensation))
    jump_term = lam*(np.exp(complex(-0.5*b*b*u*u, u*a)) - 1)        
    return np.exp(lnf_factor + (drift_and_diffusion_term + jump_term)*t )
    
def carr_madan_MJD_call_option(forward, vol, a, b, lam, t, k, r, alpha=1.5):    
    cf = lambda u: MDJ_cf(forward, vol, a, b, lam, t, u)
    integrand = lambda u: carr_madan_integrand(cf, np.log(k), u, alpha)

    # lower limit is 0.0, upper limit is infinity
    result, error = integrate.quad(integrand, 0.0, np.inf)
    return np.exp(-r*t) * np.exp(-alpha*np.log(k)) / np.pi * result

### Different maturities

In [3]:
S0 = 100
T = 1
r = 0.05
q = 0.03
vol = 0.5; lam = 1; a = -0.3; b = 0.2
Ks = np.linspace(30, 170, 15)

df = pd.DataFrame()
df['Strike'] = Ks
  
DF = np.exp(-r*T)
DivF = np.exp(-q*T)
forward = S0 * DivF / DF

callOrPut = OptionTypes.EUROPEAN_CALL

start = time.time()
my_price = [mjd.QF602MJDCall(S0, K, r, q, vol, a, b, lam, T) for K in Ks]
end = time.time()
print(end - start)

start = time.time()
his_price = [mjd.MertonCallPrice(S0, K, r, q, vol, a, b, lam, T) for K in Ks]
end = time.time()
print(end - start)

start = time.time()
fourier_price = [carr_madan_MJD_call_option(forward, vol, a, b, lam, T, K, r) for K in Ks]
end = time.time()
print(end - start)

my_ivols = [mjd.ImpliedVolatility(call_value, K, T, S0, r, q) for call_value, K in zip(my_price, Ks)] 
his_ivols = [mjd.ImpliedVolatility(call_value,K,T,S0,r,q) for call_value, K in zip(his_price, Ks)] 
fourier_ivols = [mjd.ImpliedVolatility(call_value,K,T,S0,r,q) for call_value, K in zip(fourier_price, Ks)] 

df['my price'] = np.array(my_price)
df['his price'] = np.array(his_price)
df['fourier'] = np.array(fourier_price)

df['my ivol'] = np.array(my_ivols)
df['his ivol'] = np.array(his_ivols)
df['fourier ivol'] = np.array(fourier_ivols)
           
df['my price % error'] = np.array(my_price)/np.array(fourier_price) - 1
df['his price % error'] = np.array(his_price)/np.array(fourier_price) - 1

df

0.023660659790039062
0.02025008201599121
0.009363174438476562


,Strike,my price,his price,fourier,my ivol,his ivol,fourier ivol,my price % error,his price % error
0,30.0,68.870193,68.870193,68.870193,0.644967,0.644967,0.644967,-3.552714e-15,-2.886580e-15
1,40.0,60.119537,60.119537,60.119537,0.630918,0.630918,0.630918,1.554312e-15,1.998401e-15
2,50.0,52.020486,52.020486,52.020486,0.620359,0.620359,0.620359,2.442491e-15,3.108624e-15
3,60.0,44.690927,44.690927,44.690927,0.612060,0.612060,0.612060,2.220446e-16,8.881784e-16
4,70.0,38.178656,38.178656,38.178656,0.605336,0.605336,0.605336,1.332268e-15,1.998401e-15
5,80.0,32.476430,32.476430,32.476430,0.599765,0.599765,0.599765,6.661338e-16,1.332268e-15
6,90.0,27.539927,27.539927,27.539927,0.595064,0.595064,0.595064,1.998401e-15,2.220446e-15
7,100.0,23.303391,23.303391,23.303391,0.591040,0.591040,0.591040,2.220446e-15,2.664535e-15
8,110.0,19.691407,19.691407,19.691407,0.587551,0.587551,0.587551,2.442491e-15,3.330669e-15
9,120.0,16.626881,16.626881,16.626881,0.584496,0.584496,0.584496,1.554312e-15,2.442491e-15
